# 1: The Fermi-LAT data

## Introduction

Hello Fermi initiate! 

In these tutorials I'm going to take you through your first Fermi-LAT analysis. 

We will look at:

* What Fermi data looks like.
* What the tools are, and what they do.
* The Python interface to the tools.
* Using a python package called Fermipy which will automate a lot of this for you.
* Some more advanced point source analysis techniques.
* Some other useful sources of information passed down through many generations of PhD students here in the group.
* Some common (and highly irritating) errors with the Fermi software, and how to solve them.

### Acknowledgements

These tutorials are very much based on the earlier set of tutorials by Dr Jamie Graham. In some places this is almost word for word identical (as users of Linux know, if something works, don't change it!). In addition, the absolutely incredible Fermi team at NASA and the University of Delaware (Liz, Julie, Jamie Holder, Jeremy), and also the work of Durham Alumnus Dr Tom Armstrong. 

"*Dale a tu cuerpo alegría Macarena,*

*Que tu cuerpo es pa' darle alegría why cosa buena,*

*Dale a tu cuerpo alegría, Macarena,*

*Hey Macarena* "

### How do we get the Fermi data?

The LAT data is stored at https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi, and the page looks like this. 

![Data Server](img/LATData.png)

We're going to be looking at a particularly interesting source of $\gamma$-rays, a reasonably bright and highly variable radio galaxy called NGC 1275 (studied in depth by Dr Anthony Brown and Dr Cameron Rulten here in our group). You can read Anthony's paper here: https://arxiv.org/abs/1101.2687 but it isn't essential reading for our tutorial, just a good example of a Fermi paper. 

The query I made to the data server in order to get this data is saved in the file "Data_Query.dat". This contains:

In [1]:
!cat Data_Query.dat

Equatorial coordinates (degrees)	(49.9507,41.5117)
Time range (MET)	(283996802,315532802)
Time range (Gregorian)	(2010-01-01 00:00:00,2011-01-01 00:00:00)
Energy range (MeV)	(100,300000)
Search radius (degrees)	15


We can see we have to make a few choices in order to get our data from the server. Lets take a look at what we chose, and why.

### Coordinates
The equatorial coordinates simply refer to the right ascension (RA) and declination of our target, NGC 1275 (if you're usnsure on RA and Dec... look here: http://curious.astro.cornell.edu/about-us/112-observational-astronomy/stargazing/technical-questions/699-what-are-ra-and-dec-intermediate).
Note our RA is in degrees (it's simpler this way, trust me). 

You can also use Galactic coordinates for this choice, which is particularly convinient if you're looking at $\gamma$-ray sources on the Galactic plane.

### Time Range
Fermi launched in 2008, and has sprent most of it's time observing the whole sky, hence there is now about 13 years of public data out there. Generally speaking, the more data we have in our analysis, the more photons we have, and often the better our results will be. That said, with variable sources (such as NGC 1275), source properties change over time, and we must investigate how this happens, but more on that later. For our study here, we have 1 year of data, to save on file space and computation time. You will find that a decade long analysis can take quite a while. My record for a single analysis is 3 days.

We define time with Fermi in 3 ways. We have the Gregorian calendar, which is the calendar we use on a day to day basis. We also have the Modified Julian Calendar, which counts days in MJD. Finally, we have Mission Elapsed Time (MET). MET is expresed in seconds since a particular date in 2001, and is the preferred way to express Fermi time.

There is a tool to convert between the three at the below link, but in general we should use MET.
https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/xTime/xTime.pl

When we look at the time range in our Query, we see it runs from 2010-2011, even though I input it in MET. The data query always comes out in Gregorian dates, as well as MET.

### Energy Range

Fermi has an effective energy range of 100 MeV to 500 GeV, which is generally referred to as the 'high energy' $\gamma$-ray band. We say it's an effective energy range, because it does detect photons down to 30 MeV, and up to approximately 1 TeV, although these higher energy photons are very rare. 

As a result, we tend to use this 100 MeV to 500 GeV range, this allows us to explore the spectrum of a source to its fullest.

### Search Radius

Our search radius corresponds to the size of the area we want to look at.

We use a larger (20+ degrees) region of interest (ROI) if:

- Bright sources near your ROI. This can allow flux from a source outside the ROI to work its way into your model. We'll explore this in detail when we come to likelihood fitting.
- Working on (or close to) the galactic plane. There are lots of bright sources near here, and the best fit of the interstellar emission model will come from a large ROI.
- Working with sources that peak towards lower energies. The point spread function (PSF) of the *Fermi-LAT* is **very** energy dependent. Its 68% containment region goes from several degrees at 100 MeV to 0.1 degrees at energies > 1 GeV. This can be rather frustrating to deal with, and is the principle reason we use likelihood fitting techniques.

And a smaller (10 - 20 degrees) ROI if

- If you're working on an AGN off the plane, without anything bright nearby.
- You're working at high energies (>1 GeV). The *Fermi* PSF allows you to cut down your ROI in this case.


### What does our data look like?

If we take a look in the data directory, there are a few files:


In [2]:
!ls data/

L1910170847288E15477E17_PH00.fits  L1910170847288E15477E17_PH02.fits
L1910170847288E15477E17_PH01.fits  L1910170847288E15477E17_SC00.fits


At first sight, the names of these files are a bit incomprehensible. The file format is FITS, which stands for 'Flexible Image Transfer System', and is the standard way to store astronomical data. The files with 'PHXX' at the end (where XX = 00 for example), are our photon files, and these store the information about all the $\gamma$-rays detected within our time range, energy range and region of interest. The one with 'SC00' at the end is our spacecraft file, and this contains information about where the LAT was pointed as a function of time. 

Lets open up a photon file and the spacecraft file and take a look inside:

In [3]:
import astropy.io.fits as pyfits

PH_file = pyfits.open('data/L1910170847288E15477E17_PH00.fits')

SC_file = pyfits.open('data/L1910170847288E15477E17_SC00.fits')

If we print the header of the photon file, we can see all of our data query, plus some more information.

In [4]:
PH_file[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
CHECKSUM= 'aHNac9MYaEMaa9MY'   / HDU checksum updated 2019-10-17T12:47:32       
TELESCOP= 'GLAST   '           / name of telescope generating data              
INSTRUME= 'LAT     '           / name of instrument generating data             
EQUINOX =                2000. / equinox for ra and dec                         
RADECSYS= 'FK5     '           / world coord. system for this file (FK5 or FK4) 
DATE    = '2019-10-17T08:47:28' / file creation date (YYYY-MM-DDThh:mm:ss U     
DATE-OBS= '2010-01-01T00:00:

Everything looks good, apart from our time range! As we have requested quite a lot of data (a years worth), it is split into different files. PH00 gives the photons from roughly the first few months of observations, which is correct. Another curiosity is the is our telescope name: GLAST. GLAST stands for Gamma-ray Large Area Space Telescope, which is what Fermi was called in design and construction. There are references to this previous name throughout the software and data: the Fermi tools all begin with 'gt' which stands for GLAST tools. 

A FITS file will contain several "HDUs", each of which contain different information. Here we looked at HDU 0. Traditionally, this is the HDU that contains image data. However in gamma ray astronomy, we tend to count events - not look at images. So if we try to access this HDU:

In [5]:
print PH_file[0].data

None


There is nothing there. Instead all of the information is kept in HDU 1.

In [6]:
print PH_file[1].data[0]

(1130.9272, 42.842426, 55.019382, 139.62119, -3.9184806, 50.449844, 123.57222, 96.496986, 344.49359, 284002842.55815482, 2994226, 284001367, 0, array([0, 0, 0], dtype=int16), array([False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True,  True, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True], dtype=bool), array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False, False,  True], dtype=bool), 0, 61.965824067592621, 0.0, 0.0, 0.0, 0.0, 0.0)


This is just the first photon in the file. As you can see it contains a lot of information. We have no idea what all of this means currently, so let's look at the header of this HDU:

In [7]:
PH_file[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional binary table                     
NAXIS1  =                   98 / width of table in bytes                        
NAXIS2  =               928216 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group (required keyword)              
TFIELDS =                   23 / number of fields in each row                   
TTYPE1  = 'ENERGY  '           / energy of event                                
TFORM1  = 'E       '           / data format of field: 4-byte REAL              
TTYPE2  = 'RA      '           / right ascension (J2000) of event               
TFORM2  = 'E       '           / data format of field: 4-byte REAL              
TTYPE3  = 'DEC     '        

So we have a load of information. If you look at the columns labelled "TTYPE[x]", these tell you what information is kept for each of the photons. So if we want to look at the energy, RA, and Dec of the first photon, we would do the following:

In [8]:
Energy = PH_file[1].data[0]['Energy']
RA = PH_file[1].data[0]['RA']
DEC = PH_file[1].data[0]['DEC']
print(Energy, RA, DEC)

(1130.9272, 42.842426, 55.019382)


Or alternatively:

In [9]:
PH_file[1].data[0][0]

1130.9272

We can now take a quick look at the header of the spacecraft file too:

In [10]:
SC_file[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
CHECKSUM= 'mgaapeWUmeaameWU'   / HDU checksum updated 2019-10-17T12:47:33       
TELESCOP= 'GLAST   '           / name of telescope generating data              
INSTRUME= 'LAT     '           / name of instrument generating data             
EQUINOX =                2000. / equinox for ra and dec                         
RADECSYS= 'FK5     '           / world coord. system for this file (FK5 or FK4) 
DATE    = '2019-10-17T08:47:29' / file creation date (YYYY-MM-DDThh:mm:ss U     
DATE-OBS= '2010-01-01T00:00:

And we get a long list of information regarding the spacecraft, fun! It is important to have a basic understanding of the data which we are operating on, as when things go wrong, or something unexpected arises, this understanding helps us troubleshoot. 

Feel free to play about with these files as much as you want. For example look at what information is contained in the PH file, or explore the SC file in further depth. However, I'll leave that to your own curiosity. In the next tutorial, I'll move onto discussion of the Fermi Tools.